### Словарь оценочных слов 
#### перевести его в dataframe для более удобной работы

In [52]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import tqdm
import re

In [3]:
m = Mystem()

In [156]:
df = pd.read_table('rusentilex.txt', sep=',',  encoding = 'utf-8', names=['Word', 'POS', 'InitialLem', 'Sentiment','Source', 'Ambiguity'])

In [157]:
df['POS'] = df['POS'].apply(lambda x: x.strip())
df['Sentiment'] = df['Sentiment'].apply(lambda x: x.strip())
df['Source'] = df['Source'].apply(lambda x: str(x).strip())

#### Нам нужны оценночные слова, так что фильтруем по opinion

In [160]:
sent = df.loc[df['Source'] == 'opinion']

In [161]:
sent.head()

,Word,POS,InitialLem,Sentiment,Source,Ambiguity
2,абракадабра,Noun,абракадабра,negative,opinion,NaN
3,абсурд,Noun,абсурд,negative,opinion,NaN
4,абсурдность,Noun,абсурдность,negative,opinion,NaN
5,абсурдный,Adj,абсурдный,negative,opinion,NaN
6,авантюра,Noun,авантюра,negative,opinion,NaN


#### Смотрим, что у нас в seed. Пригодится для последующего анализа

In [55]:
with open('seed.txt', 'r', encoding='utf-8') as f:
    text = f.readlines()
sentiment_list = [line.strip() for line in text]
sentiment_list

['аналогов нет',
 'подошва',
 'вежливый',
 'великолепный',
 'вкусный',
 'внимательный',
 'большой выбор',
 'вау',
 'неплохой',
 'идеальный',
 'неприятный',
 'понравилось',
 'рекомендуем',
 'на ура',
 'советуем',
 'отвратительный',
 'отличный',
 'положительный',
 'отрицательный',
 'приятный',
 'напоминать о себе',
 'спасибо',
 'радовать глаз',
 'на совесть',
 'молодцы',
 'придти еще раз',
 'орать',
 'улыбаться',
 'приветливый',
 'отличный',
 'замечательный']

### Тестовый корпус
#### извлекаем из xml id отзыва, сам отзыв, оценки за еду, интерьер и сервис. Создаём из извлечённых данных dataframe

In [120]:
with open('sent_train.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    #text = text.split('</review>')
    

In [121]:
import re
id_rew = re.findall('<review id="(.+?)">', text)
reviews = re.findall('<text>(.+?)</text>', text)
food =  re.findall('<food>(.+?)</food>', text)
interior = re.findall('<interior>(.+?)</interior>', text)
service = re.findall('<service>(.+?)</service>', text)

In [62]:
with open('reviews.txt', 'a', encoding='utf-8') as f:
    for r in reviews:
        f.write(r)

In [154]:
text_score = pd.DataFrame(np.column_stack([id_rew,reviews, food, interior, service]), 
                               columns=['id','review', 'food', 'interior', 'service'])

In [155]:
text_score.head()

,id,review,food,interior,service
0,17600,И пускай на меня не обижается наш прославленны...,8,8,8
1,23518,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,...",9,7,10
2,27221,"Советую вам уволить Вашего метродотеля Елену, ...",9,9,1
3,29097,отличный средне вековый интеръер. Приятное обс...,8,10,9
4,23065,Ужинали в ресторане Баден-Баден 6 марта . Импо...,10,8,8


#### Лемматизируем, переводим оценки в int

In [158]:
text_score['review'] = text_score['review'].apply(lambda x: m.lemmatize(x))

In [159]:
text_score['food'] = text_score['food'].apply(lambda x: int(x))
text_score['interior'] = text_score['interior'].apply(lambda x: int(x))
text_score['service'] = text_score['service'].apply(lambda x: int(x))

In [164]:
text_score['review'] = text_score['review'].apply(lambda x: ''.join(x))

#### Делим условно отзывы на позитивные и негативные по оценке за еду. Всё, что между 4 и 7, считаем нейтральными

In [163]:
pos = text_score.loc[text_score['food'] > 7]
neg = text_score.loc[text_score['food'] < 4]

#### Сохраняем в csv нашу таблицу

In [165]:
text_score.to_csv('reviews_score.csv', sep=';',encoding='utf-8', header=True)

In [5]:
df = pd.read_csv('reviews_score.csv', sep=';',  encoding = 'utf-8')

### W2V model

In [66]:
text = '.'.join(df['review'])
sentences_list = [x.strip() for x in text.split('.')]

In [68]:
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

In [69]:
words_sentences_list = [x.split(' ') for x in sentences_list]

for i in range(len(words_sentences_list)):
    for j in range(len(words_sentences_list[i])):
        words_sentences_list[i][j] = preprocessing(words_sentences_list[i][j])

words_sentences_list = [word for word in words_sentences_list if word != ' ']

In [70]:
import gensim
model = gensim.models.Word2Vec(words_sentences_list, size=500, window=10, min_count=2, sg=0)

### Topic modeling

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from nltk.corpus import stopwords

In [72]:
df.head()

,Unnamed: 0,id,review,food,interior,service
0,0,17600,и пускай на я не обижаться наш прославленный з...,8,8,8
1,1,23518,- здравствовать. виа д’арженто! - добрый вечер...,9,7,10
2,2,27221,"советовать вы увольнять ваш метродотель елена,...",9,9,1
3,3,29097,отличный средне вековый интеръер. приятный обс...,8,10,9
4,4,23065,ужинать в ресторан баден-баден 6 март . импоза...,10,8,8


In [73]:
text = df['review'].tolist()

In [28]:
vectorizer = TfidfVectorizer(min_df = 10, stop_words=stopwords.words('russian'))
A = vectorizer.fit_transform(text)
terms = vectorizer.get_feature_names()

#### Get descriptors with NMF

In [29]:
k = 25
model = NMF( init="nndsvd", n_components=k) 

W = model.fit_transform( A )
H = model.components_

In [32]:
# show topic descriptors
def get_descriptor( terms, H, topic_index, top ):
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( terms[term_index] )
    return top_terms

descriptors = []
for topic_index in range(k):
    descriptors.append( get_descriptor( terms, H, topic_index, 20 ) )
    str_descriptor = ", ".join( descriptors[topic_index] )
    print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )

Topic 01: салат, соус, суп, блюдо, вкус, мясо, меню, десерт, овощ, заказывать, чай, тарелка, большой, сыр, вино, горячий, креветка, лосось, кусочек, порция
Topic 02: quot, вопрос, название, цезарь, сегодня, слово, ответ, официантка, который, спрашивать, становиться, меню, отвечать, оценка, русский, новый, палочка, получать, коктейль, чебурек
Topic 03: очень, понравиться, обязательно, долго, интерьер, особенно, уютно, официант, прийти, милый, порадовать, внимательный, приветливый, обслуживать, красивый, единственный, нравиться, вежливый, подруга, муж
Topic 04: свадьба, спасибо, наш, гость, весь, банкет, праздник, зал, огромный, отмечать, отдельный, администратор, благодарность, торт, персонал, праздновать, банкетный, организация, свой, торжество
Topic 05: это, мочь, сказать, который, весь, человек, свой, стол, знать, понимать, ничто, вообще, девушка, хотя, зал, говорить, кафе, самый, сидеть, время
Topic 06: приносить, минута, заказ, ждать, официант, заказывать, столик, официантка, час, 

#### to make words searchable in W2V model

In [133]:
with open('SEED.txt', 'r', encoding='utf-8') as f:
    golden = f.readlines()

In [134]:
golden = [i.strip() for i in golden]

In [122]:
topic_descriptors = [item for sublist in descriptors[1:] for item in sublist]

In [40]:
## ищем векторы для seed, ищем векторы для дескрипторов
## ищем по дескрипторам наиболее похожие к сиду, а потом ищем по всей модельки более похожие и делаем список

In [144]:
new_golden = []

In [145]:
len(topic_descriptors)

480

In [146]:
for g in golden:
    if model.__contains__(g) is True:
        for t in topic_descriptors:
            if model.__contains__(t) is True:
                if model.wv.similarity(g, t) > 0.6:
                    new_golden.append(t)
            else:
                print('Sorry')
        new_golden.append(g)
    else:
                print('Sorry')

/Users/Inga/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/Users/Inga/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.


Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry
Sorry


In [147]:
new_golden = set(new_golden)

In [148]:
len(new_golden)

57

In [149]:
new_golden

{'администратор',
 'бармен',
 'благодарность',
 'вау',
 'вежливый',
 'великолепный',
 'вкусно',
 'вкусный',
 'внимательный',
 'впечатление',
 'девушка',
 'достойный',
 'живой',
 'замечательный',
 'играть',
 'идеальный',
 'китайский',
 'классный',
 'красивый',
 'милый',
 'молодец',
 'ненавязчивый',
 'неплохой',
 'неприятный',
 'нормальный',
 'обслуживание',
 'обслуживать',
 'объедение',
 'орать',
 'организация',
 'отвечать',
 'отвратительный',
 'отличный',
 'отменный',
 'отрицательный',
 'официант',
 'официантка',
 'персонал',
 'пиццерия',
 'плохой',
 'положительный',
 'понравилось',
 'постоянно',
 'потрясать',
 'превосходный',
 'прекрасный',
 'приветливый',
 'приемлемый',
 'приятный',
 'разнообразный',
 'спасибо',
 'супер',
 'улыбаться',
 'уютный',
 'хата',
 'хороший',
 'японский'}

In [ ]:
for word in words:
    # есть ли слово в модели? Может быть, и нет
    if word in model:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print(word + ' is not present in the model')

In [179]:
sentiment_list = {}

In [180]:
for word in new_golden:
    for i in model.most_similar(positive=[word], topn=5):
            # слово + коэффициент косинусной близости
            #print(word, i[0], i[1])
            sentiment_list[i[0]]=i[1]#sentiment_list.append(i[0])
    #print('\n')

/Users/Inga/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [181]:
with open('sentiment_list.txt', 'a', encoding='utf-8') as f:
    for i in set(sentiment_list):
        f.write(i + '\t' + str(sentiment_list[i]) + '\n')